In [1]:
import pandas as pd

### Cache data

In [3]:
from data.data import PriceData, ProductData, BenchmarkData
from config import Benchmarks, START_DATE, END_DATE

In [ ]:
benchmark_data = BenchmarkData()
tickers = benchmark_data.get_constituents(Benchmarks.SP500)
len(tickers)

In [ ]:
product_data = ProductData()
data = product_data.get_data(tickers=tickers)
len(data)

In [ ]:
price_data = PriceData()
data = price_data.get_data(tickers, start_date=START_DATE, end_date='2025-06-01')
len(data)

### Cache portfolio 

In [2]:
from backtesting.portfolio_cache import create_and_cache_portfolio
from config import Benchmarks, Strategies, INITIAL_SETUP, DEFAULT_BACKTEST_PARAMS

In [ ]:
portfolio, cache_key = create_and_cache_portfolio(
    portfolio_name="SP500_2025_Jun_10yrs",
    benchmark=Benchmarks.SP500,
    start_date="2015-01-01",
    end_date="2025-06-01",
    strategies=[
        Strategies.MACD_CROSSOVER,
        Strategies.RSI_CROSSOVER,
        Strategies.BOLLINGER_BANDS,
        Strategies.Z_SCORE_MEAN_REVERSION,
    ],
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    setup=INITIAL_SETUP,
    description="Selected 10 yrs SP500 portfolio using all 4 strategies",
)
cache_key

### Generate report

In [1]:
from backtesting.portfolio_cache import load_cached_portfolio
from reporting.report_generator import ReportGenerator
from IPython.display import Image, display
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import os

In [ ]:
cached_key = "SP500_2025_Jun_10yrs_sp500_2015_01_01_2025_06_01_macd_crossover_rsi_crossover_bollinger_bands_z_score_mean_reversion_constraints__8001337252388095717_setup__4724061233836950910"
portfolio = load_cached_portfolio(cached_key)
portfolio.generate_report()

Portfolio loaded from cache: SP500_2025_Jun_10yrs_sp500_2015_01_01_2025_06_01_macd_crossover_rsi_crossover_bollinger_bands_z_score_mean_reversion_constraints__8001337252388095717_setup__4724061233836950910
Custom PDF report saved to: reports/SP500_2025_Jun_10yrs_custom_report_20250605_163303.pdf


'reports/SP500_2025_Jun_10yrs_custom_report_20250605_163303.pdf'

In [ ]:
portfolio.generate_analytics()
metrics = portfolio.metrics
holdings_summary = portfolio.holdings_summary
report_generator = ReportGenerator(portfolio, metrics, holdings_summary)

In [ ]:
def show_me(data, filename="temp"):
    c = canvas.Canvas(f"temp/{filename}.pdf", pagesize=letter)
    data.wrapOn(c, 400, 300)
    data.drawOn(c, 50, 500)
    c.save()
    os.system(f"sips -s format png temp/{filename}.pdf --out temp/{filename}.png")  # Mac
    display(Image(f"temp/{filename}.png"))

In [ ]:
# table = report_generator.create_holdings_summary_table()
# show_me(table)

In [ ]:
# buf = report_generator.create_return_distribution_charts()
# display(Image(buf.getvalue())), buf

### Backtesting

In [4]:
from portfolio.portfolio import Portfolio
from backtesting.backtest import Backtest
from config import DEFAULT_BACKTEST_PARAMS, INITIAL_SETUP, Benchmarks, Strategies

In [6]:
strategies = [Strategies.MACD_CROSSOVER]
start_date = "2025-05-01"
end_date = "2025-06-01"

portfolio = Portfolio(
    name="dummy portfolio",
    benchmark=Benchmarks.SP500,
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    additional_setup=INITIAL_SETUP,
)
backtest = Backtest(
    portfolio=portfolio, start_date=start_date, end_date=end_date, strategies=strategies
)

backtest.run()
backtest.generate_report("dummy_backtest.pdf")

Backtest starting... whomp whomp!
Total trading days: 22 
. Starting in 2025-05-01.


Backtesting: 100%|██████████| 22/22 [00:01<00:00, 15.37day/s]


Backtest completed!
Custom PDF report saved to: reports/dummy_portfolio_custom_report_20250605_163840.pdf


'reports/dummy_portfolio_custom_report_20250605_163840.pdf'